In [39]:
%%bash
docker run \
  --rm \
  --name omnisci_test \
  --ipc=host \
  -d --runtime=nvidia \
  -v $(readlink -f ..)/data:/data \
  -v $(readlink -f ..)/scripts:/src \
  -p 6273-6280:6273-6280 \
  omnisci/core-os-cuda:v5.2.1

82dd93d007a1215d96c842ce249173c6dd9d9fabebc268aef8f83153599969f2


In [40]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "cat /src/load_dataset.sql | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

User admin connected to database omnisci
Result
Loaded: 1067371 recs, Rejected: 0 recs in 0.299000 secs
1 rows returned.
Execution time: 398 ms, Total time: 398 ms
User admin disconnected from database omnisci


In [41]:
%%bash
docker exec omnisci_test /bin/bash -c \
    "echo 'SELECT * FROM online_retail LIMIT 10;' | /omnisci/bin/omnisql -p 'HyperInteractive' -t"

User admin connected to database omnisci
Invoice|StockCode|Description|Quantity|InvoiceDate|Price|CustomerID|Country
491653|22086|PAPER CHAIN KIT 50'S CHRISTMAS|8|2009-12-13 10:50:00|2.95|18077.0|United Kingdom
491653|21786|RAIN HAT WITH RED SPOTS|24|2009-12-13 10:50:00|0.42|18077.0|United Kingdom
491653|21790|VINTAGE SNAP CARDS|24|2009-12-13 10:50:00|0.85|18077.0|United Kingdom
491812|21711|FOLDING UMBRELLA  WHITE/RED  SPOT|24|2009-12-14 13:23:00|1.95|13694.0|United Kingdom
491653|21791|VINTAGE HEADS AND TAILS CARD GAME|6|2009-12-13 10:50:00|1.25|18077.0|United Kingdom
491812|21846|PINK DIAMANTE PEN IN GIFT BOX|150|2009-12-14 13:23:00|1.25|13694.0|United Kingdom
491675|21484|CHICK GREY HOT WATER BOTTLE|1|2009-12-13 12:04:00|3.45|16710.0|United Kingdom
491654|90182C|BLACK 3 BEAD DROP EARRINGS|2|2009-12-13 10:54:00|2.95|16104.0|United Kingdom
489612|22086|PAPER CHAIN KIT 50'S CHRISTMAS|6|2009-12-01 14:55:00|6.04|NULL|United Kingdom
491812|21847|GREEN  DIAMANTE PEN IN GIFT BOX|24|2009-12

In [42]:
from pymapd import connect
from time import time
import pandas as pd
import numpy as np
import json

In [43]:
with open('test_result.json') as json_file:
    test_result = json.load(json_file)

In [44]:
con = connect(user="admin", password="HyperInteractive", host="localhost", dbname="omnisci")

In [45]:
test_result["timings"]["omnisci"] = {}

for label, query in test_result["queries"].items():
    print(f"Executing {label} -> {query}")
    test_result["timings"]["omnisci"][label] = []
    for i in range(0, 30):
        start = time()
        pd.read_sql_query(query, con=con)  # <-un po' più veloce ma usa ipc (arrow)
        # df = con.select_ipc_gpu(query)  # <-analogo
        # result = list(con.execute(query))  #NO IPC
        end = time()

        total = end-start
        test_result["timings"]["omnisci"][label].append(total)

with open('test_result.json', 'w') as outfile:
    json.dump(test_result, outfile, indent=2)

Executing Q1 -> SELECT Invoice, SUM(Price*Quantity) as invoice_sum FROM online_retail GROUP BY Invoice
Executing Q2 -> SELECT CustomerID, SUM(Price*Quantity) as customer_sum FROM online_retail GROUP BY CustomerID
Executing Q3 -> SELECT Country, SUM(Price*Quantity) as country_sum FROM online_retail GROUP BY Country
Executing Q4 -> SELECT StockCode, COUNT(*) as n_items FROM online_retail GROUP BY StockCode ORDER BY n_items DESC
Executing Q5 -> SELECT Invoice, SUM(Price*Quantity) as invoice_sum FROM online_retail GROUP BY Invoice ORDER BY invoice_sum DESC
Executing Q6 -> SELECT InvoiceDate, AVG(Price*Quantity) as avg_total FROM online_retail GROUP BY InvoiceDate ORDER BY avg_total
Executing Q7 -> SELECT Invoice, SUM(Price*Quantity) as invoice_sum FROM online_retail WHERE Country='France' GROUP BY Invoice ORDER BY invoice_sum DESC
Executing Q8 -> SELECT CustomerID, Price*Quantity as total_price FROM online_retail WHERE Quantity > 20 ORDER BY total_price DESC
Executing Q9 -> SELECT Customer

## Join tests

In [10]:
query = "SELECT Invoice, stock_codes.Price FROM online_retail JOIN stock_codes ON online_retail.StockCode = stock_codes.StockCode"

start = time()
pd.read_sql_query(query, con=con)
end = time()

total = end-start
print(total)

1.6789634227752686


In [11]:
query = "SELECT Invoice, Price FROM online_retail"

start = time()
pd.read_sql_query(query, con=con)
end = time()

total = end-start
print(total)

1.4988090991973877


In [37]:
%%bash
docker stop omnisci_test

omnisci_test
